In [13]:
#################
### MLP for custom data
#################
import numpy as np
import tensorflow as tf
from sklearn import datasets, model_selection
import math


### define hyperparameters
n_classes = 3
n_features = 4
n_hidden_1 = 10
n_hidden_2 = 20
n_hidden_3 = 10
n_hidden_4 = 20

learning_rate = 0.01
max_iter = 10000

### define graph
x = tf.placeholder(tf.float32, shape=[None, n_features], name="X")
y_label = tf.placeholder(tf.int32, shape=[None], name="Y_label")
# one-hot encoding
y = tf.one_hot(indices=y_label, depth=n_classes)

# hidden 1
with tf.name_scope('hidden1'):
  weights1 = tf.Variable(
      tf.truncated_normal([n_features, n_hidden_1],
                          stddev=1.0),
      name='weights')
  biases1 = tf.Variable(
      tf.zeros([n_hidden_1]),
      name='biases')
  hidden1 = tf.nn.relu(tf.matmul(x, weights1) + biases1)
# hidden 2
with tf.name_scope('hidden2'):
  weights2 = tf.Variable(
      tf.truncated_normal([n_hidden_1, n_hidden_2],
                          stddev=1.0),
      name='weights')
  biases2 = tf.Variable(
      tf.zeros([n_hidden_2]),
      name='biases')
  hidden2 = tf.nn.relu(tf.matmul(hidden1, weights2) + biases2)
# hidden 3
with tf.name_scope('hidden3'):
  weights3 = tf.Variable(
      tf.truncated_normal([n_hidden_2, n_hidden_3],
                          stddev=1.0),
      name='weights')
  biases3 = tf.Variable(
      tf.zeros([n_hidden_3]),
      name='biases')
  hidden3 = tf.nn.relu(tf.matmul(hidden2, weights3) + biases3)
# hidden 4
with tf.name_scope('hidden4'):
  weights4 = tf.Variable(
      tf.truncated_normal([n_hidden_3, n_hidden_4],
                          stddev=1.0),
      name='weights')
  biases4 = tf.Variable(
      tf.zeros([n_hidden_4]),
      name='biases')
  hidden4 = tf.nn.relu(tf.matmul(hidden3, weights4) + biases4)
# Linear
with tf.name_scope('softmax_linear'):
  weights_1 = tf.Variable(
      tf.truncated_normal([n_hidden_2, n_classes],
                          stddev=1.0),
      name='weights')
  biases_1 = tf.Variable(
      tf.zeros([n_classes]),
      name='biases')
  logits_1 = tf.matmul(hidden2, weights_1 + biases_1)

pred1 = tf.cast( tf.argmax(logits_1, 1), tf.int32 )
accuracy1 = tf.reduce_mean( tf.cast( tf.equal(pred1, y_label), tf.float32 ))

# Linear
with tf.name_scope('softmax_linear2'):
  weights_2 = tf.Variable(
      tf.truncated_normal([n_hidden_4, n_classes],
                          stddev=1.0),
      name='weights')
  biases_2 = tf.Variable(
      tf.zeros([n_classes]),
      name='biases')
  logits_2 = tf.matmul(hidden4, weights_2 + biases_2)

pred2 = tf.cast( tf.argmax(logits_2, 1), tf.int32 )
accuracy2 = tf.reduce_mean( tf.cast( tf.equal(pred2, y_label), tf.float32 ))

### define loss
loss1 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits_1, labels=y))
loss2 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits_2, labels=y))
train_step1 = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss1)
train_step2 = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss2)





### define input data
# Load the iris dataset
iris = datasets.load_iris()

# iris has two attributes: data, target
print(iris.data.shape)
print(iris.target.shape)

# Split the data into training/testing sets
x_train, x_test, y_train, y_test = model_selection.train_test_split(
  iris.data, iris.target, test_size=0.3)



### Starting sessions
with tf.Session() as sess:
  ## initialize variables
  init = tf.global_variables_initializer()
  sess.run(init)

  ## training
  for i in range(max_iter):
    _, accuracy_val1, pred_val1, loss_val1 = sess.run(
      [train_step1, accuracy1, pred1, loss1], 
      feed_dict={x: x_train, y_label: y_train})
    
    _, accuracy_val2, pred_val2, loss_val2 = sess.run(
      [train_step2, accuracy2, pred2, loss2], 
      feed_dict={x: x_train, y_label: y_train})

    if i % 1000 == 0:
      print('=========== Epoch: %d ===========' % i)
      print('Loss', loss_val1)
      print('Loss2', loss_val2)  
      print('Accuracy', accuracy_val1)
      print('Accuracy2', accuracy_val2)
      print('Y_prediction ', pred_val1[:10])
      print('True', y_train[:10])
      
      # accuracy for testset
      test_accuracy1, test_pred1 = sess.run( 
        [accuracy1, pred1],
        feed_dict={x: x_test, y_label: y_test})
      test_accuracy2, test_pred2 = sess.run( 
        [accuracy2, pred2],
        feed_dict={x: x_test, y_label: y_test})  
      print('---- evaluation ----')
      print('acc: %.4f' % test_accuracy1)
      print('acc2: %.4f' % test_accuracy2)
      print('pred', test_pred1)
      print('pred2', test_pred2)
      print('true', y_test)


    if math.isnan(loss_val):
      print('LOSS is NAN!')
      break

(150, 4)
(150,)
=========== Epoch: 0 ===========
('Loss', 34.562725)
('Loss2', 117.05624)
('Accuracy', 0.33333334)
('Accuracy2', 0.33333334)
('Y_prediction ', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32))
('True', array([1, 0, 1, 1, 2, 0, 0, 0, 1, 2]))
---- evaluation ----
acc: 0.3333
acc2: 0.3333
('pred', array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32))
('pred2', array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32))
('true', array([2, 0, 1, 1, 2, 1, 1, 0, 1, 0, 2, 0, 2, 1, 0, 0, 0, 2, 2, 0, 1, 2, 2,
       1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 2, 0, 2, 1, 2, 1, 1, 2, 0, 2, 2, 2]))
=========== Epoch: 1000 ===========
('Loss', 1.098612)
('Loss2', 1.098612)
('Accuracy', 0.33333334)
('Accuracy2', 0.33333334)
('Y_prediction ', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=